In [ ]:
import numpy as np
import nbimporter
import m00_helper as helper
from typing import Callable #to type hint Callable functions

#### Create profiles of agents' preferences over issues

In [ ]:
def issues_profile(n_agents, n_issues, p, seed=None)->np.ndarray:
    '''returns a binary np.ndarray of shape n_agents x n_issues with vals drawn from Bernoulli with param p'''
    np.random.seed(seed)
    return np.random.binomial(1, p, size=(n_agents, n_issues))

#### Compute the distances between voters and cands based on their prefereces

- Distances are normalized to be in the inclusive range [0,1] where 0 means identical preferences over issues and 1 means they maximally disagree on the issues

In [ ]:
def issues_to_distances(voter_issues_profile, cand_issues_profile)->np.ndarray:
    '''returns pairwise distances between voters and cands from their prefs over issues as matrix of size n_voters x n_cands'''
    hamming_distances = np.sum(voter_issues_profile[:, None] != cand_issues_profile, axis=2)
    return hamming_distances / voter_issues_profile.shape[1] #normalize distances by dividing by num issues

#### Compute voter prefs over candidates induced by distances between them

In [ ]:
def distances_to_threshold_approvals(voter_cand_distances, threshold)->np.ndarray:
    '''returns matrix of approval ballots of voters over candidates based on if distance is strictly below a threshold'''
    return voter_cand_distances < threshold

In [ ]:
def distance_to_k_approval_one_voter(distances, k, seed = None):
    '''Given a 1D np.ndarray returns the k smallest values with ties broken randomly among those tied for k^th'''
    distances_augmented = helper.array1D_to_sorted_indices(distances, seed)
    approved_cands = distances_augmented[:,2][:k].astype(int)
    indicator_array = np.zeros(len(distances), dtype=int)
    indicator_array[approved_cands] = 1
    return indicator_array

In [ ]:
def distances_to_k_approvals(voter_cand_distances, k, seed=None)->np.ndarray:
    '''returns matrix of approval ballots where voters approve of top k cands with lowest distance to them.
        Ties are broken randomly.
    '''
    return np.array(list(map(distance_to_k_approval_one_voter,
                             voter_cand_distances, 
                             [k]*len(voter_cand_distances),
                            [seed]*len(voter_cand_distances))))

In [ ]:
def distances_to_scores(voter_cand_distances)->np.ndarray:
    '''returns matrix of scores from voters of candidates based on matrix of distances'''
    return 1-voter_cand_distances

In [ ]:
def distances_to_ordermaps_one_voter(distances, seed = None):
    ordermap = np.empty_like(distances, dtype=int)
    distances_augmented = helper.array1D_to_sorted_indices(distances, seed)
    sorted_indices = distances_augmented[:,2].astype(int)
    for idx, elem in enumerate(sorted_indices):
        ordermap[elem] = idx
    return ordermap

In [ ]:
def distances_to_ordermaps(voter_cand_distances, seed = None)->np.ndarray:
    '''returns matrix of n_voters x n_cands where M[i,j] is the rank of cand j according to voter i.
        Ties are broken randomly.
    '''
    return np.array(list(map(distances_to_ordermaps_one_voter,
                             voter_cand_distances,
                             [seed]*voter_cand_distances.shape[0])))

In [ ]:
def distances_to_orders_one_voter(distances, seed = None):
    ordermap = np.empty_like(distances, dtype=int)
    distances_augmented = helper.array1D_to_sorted_indices(distances, seed)
    sorted_indices = distances_augmented[:,2].astype(int)
    return sorted_indices

In [ ]:
def distances_to_orders(voter_cand_distances, seed=None)->np.ndarray:
    '''returns matrix of with orders by voters over candidates based on distances.
        Ties are broken randomly
    '''
    return np.array(list(map(distances_to_orders_one_voter,
                             voter_cand_distances,
                             [seed]*voter_cand_distances.shape[0])))


### Convenience Functions

In [ ]:
def generate_instances(n_voters, n_cands, n_issues, voters_p, cands_p, n_instances, start_seed=0):
    instance = {}
    for i in range(n_instances):
        voter_profile = issues_profile(n_voters, n_issues, voters_p, seed=start_seed+i)
        cand_profile = issues_profile(n_cands, n_issues, cands_p, seed=start_seed+i)
        distances = issues_to_distances(voter_profile, cand_profile)
        instance['voter_issues_profile'] = voter_profile
        instance['cand_issues_profile'] = cand_profile
        instance['voter_cand_distances'] = distances
        yield instance

In [ ]:
def all_issues_profile_derivatives(voter_issues_profile, cand_issues_profile, threshold, k):
    profile_derivatives = {}
    voter_cand_distances = issues_to_distances(voter_issues_profile, cand_issues_profile)
    profile_derivatives['voter_cand_distances'] = voter_cand_distances
    profile_derivatives['threshold_approval_profile'] = distances_to_threshold_approvals(voter_cand_distances, threshold)
    profile_derivatives['k_approval_profile'] = distances_to_k_approvals(voter_cand_distances, k)
    profile_derivatives['orders_profile'] = distances_to_orders(voter_cand_distances)
    profile_derivatives['ordermaps_profile'] = distances_to_ordermaps(voter_cand_distances)
    profile_derivatives['scores_profile'] = distances_to_scores(voter_cand_distances)
    return profile_derivatives

### Quick Tests

In [ ]:
N_VOTERS = 2
N_CANDS = 3
N_ISSUES = 3

voter_profile = issues_profile(N_VOTERS, N_ISSUES, p=0.5)
print(f'\nvoter profile:\n{voter_profile}')
cands_profile = issues_profile(N_CANDS, N_ISSUES, p=0.5)
print(f'\ncands profile:\n{cands_profile}')
distances = issues_to_distances(voter_profile, cands_profile)
print(f'\ndistances:\n{distances}')
thresh_approvals = distances_to_threshold_approvals(distances, 0.5)
print(f'\nthreshold approvals:\n{thresh_approvals}')
# k_approvals = distance_to_k_approval_one_voter(distances[0], 1)
# print(f'1_approvals: {k_approvals}')
# k_approvals = distance_to_k_approval_one_voter(distances[0], 2)
# print(f'2_approvals: {k_approvals}')
# k_approvals = distance_to_k_approval_one_voter(distances[0], 3)
# print(f'3_approvals: {k_approvals}')
ordermaps = distances_to_ordermaps(distances)
print(f'\nordermaps:\n{ordermaps}')
orders = distances_to_orders(distances)
print(f'\norders:\n{orders}')
# k_approvals = distances_to_k_approvals(distances, 2)
# print(f'2-approvals: {k_approvals}')
scores = distances_to_scores(distances)
print(f'\nscores:\n{scores}')

derivatives = all_issues_profile_derivatives(voter_profile, cands_profile, 0.5, 2)
print()
print(derivatives)


voter profile:
[[0 0 0]
 [1 0 0]]

cands profile:
[[1 1 0]
 [1 1 1]
 [0 0 1]]

distances:
[[0.66666667 1.         0.33333333]
 [0.33333333 0.66666667 0.66666667]]

threshold approvals:
[[False False  True]
 [ True False False]]

ordermaps:
[[1 2 0]
 [0 2 1]]

orders:
[[2 0 1]
 [0 2 1]]

scores:
[[0.33333333 0.         0.66666667]
 [0.66666667 0.33333333 0.33333333]]

{'voter_cand_distances': array([[0.66666667, 1.        , 0.33333333],
       [0.33333333, 0.66666667, 0.66666667]]), 'threshold_approval_profile': array([[False, False,  True],
       [ True, False, False]]), 'k_approval_profile': array([[1, 0, 1],
       [1, 0, 1]]), 'orders_profile': array([[2, 0, 1],
       [0, 1, 2]]), 'ordermaps_profile': array([[1, 2, 0],
       [0, 1, 2]]), 'scores_profile': array([[0.33333333, 0.        , 0.66666667],
       [0.66666667, 0.33333333, 0.33333333]])}
